<div style="background: #000;
            color: #FFF;
            margin: 0px;
            padding: 10px 0px 20px 0px;
            text-align: center; 
                ">
    <h1>Week 18 Lab</h1>
</div>

For today's lab, we'll be using the Marvel comic book characters data set available from [538](https://github.com/fivethirtyeight/data/tree/master/comic-characters)

The data comes from [Marvel Wikia](http://marvel.wikia.com/Main_Page) and [DC Wikia](http://dc.wikia.com/wiki/Main_Page). Characters were scraped on August 24. Appearance counts were scraped on September 2. The month and year of the first issue each character appeared in was pulled on October 6.

The data is split into two files, for DC and Marvel, respectively: `dc-wikia-data.csv` and `marvel-wikia-data.csv`. Each file has the following variables:

Variable | Definition
---|---------
`page_id` | The unique identifier for that characters page within the wikia
`name` | The name of the character
`urlslug` | The unique url within the wikia that takes you to the character
`ID` | The identity status of the character (Secret Identity, Public identity, [on marvel only: No Dual Identity])
`ALIGN` | If the character is Good, Bad or Neutral
`EYE` | Eye color of the character
`HAIR` | Hair color of the character
`SEX` | Sex of the character (e.g. Male, Female, etc.)
`GSM` | If the character is a gender or sexual minority (e.g. Homosexual characters, bisexual characters)
`ALIVE` | If the character is alive or deceased
`APPEARANCES` | The number of appareances of the character in comic books (as of Sep. 2, 2014. Number will become increasingly out of date as time goes on.)
`FIRST APPEARANCE` | The month and year of the character's first appearance in a comic book, if available
`YEAR` | The year of the character's first appearance in a comic book, if available


Importing the libraries we'll be using

In [120]:
import numpy as np 
import pandas as pd 
import datetime

Let's open the `marvel-wikia-data.csv` file and generate a dataframe. Then we'll check the top of the `df` to see what we have.

In [121]:
df = pd.read_csv("../datasets/marvel-wikia-data.csv")
df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0


As you can see we have a couple of issues:
* The column `names` has, in some cases, both the hero name and their secret identity. In other cases, it has the universe the hero is from (usually Earth-616). We'd like for `name` to be the superhero. If a secret identity name is available, we'd like for that to be in it's own column. We'd also like a column `origin` for the superhero's place of origin (usually Earth-616).
* The `urlslug` column is incorrect. We'd like for that to be a complete link. For example, spiderman's wikia link is https://marvel.fandom.com/wiki/Peter_Parker_(Earth-616) but if you use the `Spider-Man_(Peter_Parker)` from the `urlslug` (https://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker), the page redirects to the correct page. You can choose to fix it various different ways but it should be a working link. Use "https://marvel.fandom.com/wiki/" as the beginning of your url.
* For the column `align`, we'd like for the "Characters" part to be removed. We'd like for that column to be categorical (Good, Neutral, Evil or blank).
* Same for the columns `EYE`, `HAIR`, `SEX` and `ID`. We want to drop the extraneous data and have simple categories (e.g. for `ID` we want categories ["Known","Public", "Secret", "None"] and for `SEX` we just want ["Male", "Female","Genderfluid","Agender"]. There's a way shown below that shows the various possible categories.
* Some data is in the wrong format. For example, `Year` and `APPEARANCES` are floats but they really should be ints (as you cannot have a non-int number of appearances). 
* Some data doesn't match up (like the `FIRST APPEARANCE` and `Year` columns might have not be the same year)
* Some columns are unnecessary. Some are missing data. Some rows are duplicates. Fix those as necessary.
* Some column names are written as uppercase, some are lowercase and some are mixed case. Normalize them so all the column names are in the same format.

Fix as many of these issues as you're able to. At the minimum, complete 4 of the above bulletpoints.

In [122]:
def see_possible_categories(col):
    return df.groupby(col).sum()
    
see_possible_categories('name')

,page_id,APPEARANCES,Year
name,,,
'Spinner (Earth-616),322600,0.0,2007.0
107 (Earth-616),116257,1.0,1994.0
11-Ball (Earth-616),543479,1.0,1991.0
115 (Legion Personality) (Earth-616),624448,1.0,2010.0
14 (Earth-616),116253,1.0,1994.0
...,...,...,...
Zzxz (Earth-616),250460,6.0,2009.0
Zzzax (Earth-616),1553,32.0,1973.0
"\""Spider-Girl\"" (Mutant\/Spider Clone) (Earth-616)",689359,0.0,2013.0


In [123]:
df["name"]

0                  Spider-Man (Peter Parker)
1            Captain America (Steven Rogers)
2        Wolverine (James \"Logan\" Howlett)
3          Iron Man (Anthony \"Tony\" Stark)
4                        Thor (Thor Odinson)
                        ...                 
16371                     Ru'ach (Earth-616)
16372        Thane (Thanos' son) (Earth-616)
16373          Tinkerer (Skrull) (Earth-616)
16374         TK421 (Spiderling) (Earth-616)
16375                  Yologarch (Earth-616)
Name: name, Length: 16376, dtype: object

In [124]:
df["name"] = df["name"].str.split('(',1).str[0]
df["name"][2]

'Wolverine '

* For the column `align`, we'd like for the "Characters" part to be removed. We'd like for that column to be categorical (Good, Neutral, Evil or blank).

In [125]:
df["align"] = df["align"].str.split('C',1).str[0]
df["align"] 

KeyError: 'align'

* Some column names are written as uppercase, some are lowercase and some are mixed case. Normalize them so all the column names are in the same format.

In [102]:
df.columns = map(str.lower, df.columns)

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16376 entries, 0 to 16375
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   page_id           16376 non-null  int64  
 1   name              16376 non-null  object 
 2   urlslug           16376 non-null  object 
 3   id                12606 non-null  object 
 4   align             13564 non-null  object 
 5   eye               6609 non-null   object 
 6   hair              12112 non-null  object 
 7   sex               15522 non-null  object 
 8   gsm               90 non-null     object 
 9   alive             16373 non-null  object 
 10  appearances       15280 non-null  float64
 11  first appearance  15561 non-null  object 
 12  year              15561 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 1.6+ MB


Some data is in the wrong format. For example, Year and APPEARANCES are floats but they really should be ints (as you cannot have a non-int number of appearances).

In [104]:
df['year'] = df['year'][df['year'].isna()]=0
df['appearances'] = df['appearances'][df['appearances'].isna()]=0
df['year']

0        0
1        0
2        0
3        0
4        0
        ..
16371    0
16372    0
16373    0
16374    0
16375    0
Name: year, Length: 16376, dtype: int64

In [105]:
df['appearances']

0        0
1        0
2        0
3        0
4        0
        ..
16371    0
16372    0
16373    0
16374    0
16375    0
Name: appearances, Length: 16376, dtype: int64

In [106]:

df['year'] = df.astype({"year":'int'})
df['year']

0          1678
1          7139
2         64786
3          1868
4          2460
          ...  
16371    657508
16372    665474
16373    695217
16374    708811
16375    673702
Name: year, Length: 16376, dtype: object

In [107]:
df['appearances'] = df.astype({"appearances":'int'})
df['appearances']

0          1678
1          7139
2         64786
3          1868
4          2460
          ...  
16371    657508
16372    665474
16373    695217
16374    708811
16375    673702
Name: appearances, Length: 16376, dtype: object

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16376 entries, 0 to 16375
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   page_id           16376 non-null  int64  
 1   name              16376 non-null  object 
 2   urlslug           16376 non-null  object 
 3   id                12606 non-null  object 
 4   align             13564 non-null  object 
 5   eye               6609 non-null   object 
 6   hair              12112 non-null  object 
 7   sex               15522 non-null  object 
 8   gsm               90 non-null     object 
 9   alive             16373 non-null  object 
 10  appearances       15280 non-null  float64
 11  first appearance  15561 non-null  object 
 12  year              15561 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 1.6+ MB
